In [1]:
%pip install -qU \
    pinecone-datasets==0.7.0 \
    pinecone-client==3.1.0 \
    pinecone-text \
    datasets \
    transformers \
    sentence-transformers \
    git+https://github.com/naver/splade.git

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
hydra-core 1.3.2 requires antlr4-python3-runtime==4.9.*, but you have antlr4-python3-runtime 4.8 which is incompatible.
hydra-core 1.3.2 requires omegaconf<2.4,>=2.2, but you have omegaconf 2.1.2 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd

data_df = pd.DataFrame({
    'id': [1, 2, 3, 4, 5],
    'word1': ['house', 'big', 'car', 'happy', 'fast'],
    'word2': ['home', 'large', 'bicycle', 'glad', 'rapid'],
    'is it synonym': [True, True, False, True, True]
})

# Accessing the first row using iloc
row_0 = data_df.iloc[0]
id_0 = row_0['id']
word1_0 = row_0['word1']
word2_0 = row_0['word2']
is_synonym_0 = row_0['is it synonym']

print(id_0, word1_0, word2_0, is_synonym_0)

1 house home True


In [3]:
def chunker(data_df):
    chunks = []
    for index, row in data_df.iterrows():
        chunk = f"{row['word1']} {row['word2']}."
        chunks.append(chunk)
    return chunks

result_chunks = chunker(data_df)

# Generate IDs for each word pair
ids = [f"{data_df['id'][i]}-{i}" for i in range(len(data_df))]
print(result_chunks)
print(ids)

['house home.', 'big large.', 'car bicycle.', 'happy glad.', 'fast rapid.']
['1-0', '2-1', '3-2', '4-3', '5-4']


In [4]:
data = []
for i, row in data_df.iterrows():
    chunks = [f"{row['word1']} {row['word2']}."]
    for j, synoniem in enumerate(chunks):
        data.append({
            'id': f"{row['id']}-{j}",
            'synoniem': synoniem
        })

data[:10]


[{'id': '1-0', 'synoniem': 'house home.'},
 {'id': '2-0', 'synoniem': 'big large.'},
 {'id': '3-0', 'synoniem': 'car bicycle.'},
 {'id': '4-0', 'synoniem': 'happy glad.'},
 {'id': '5-0', 'synoniem': 'fast rapid.'}]

In [5]:
from sentence_transformers import SentenceTransformer
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
# check device being run on
if device == 'cpu':
    print("==========\n"+
          "WARNING: You are not running on GPU so this may be slow.\n"+
          "If on Google Colab, go to top menu > Runtime > Change "+
          "runtime type > Hardware accelerator > 'GPU' and rerun "+
          "the notebook.\n==========")

dense_model = SentenceTransformer(
    'msmarco-bert-base-dot-v5',
    device=device
)
dense_model


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


.gitattributes:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/6.19k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

In [6]:
emb = dense_model.encode(str(data[0]['synoniem']))
emb.shape

(768,)

In [7]:
dim = dense_model.get_sentence_embedding_dimension()
dim

768

In [8]:
from splade.models.transformer_rep import Splade

sparse_model_id = 'naver/splade-cocondenser-ensembledistil'

sparse_model = Splade(sparse_model_id, agg='max')
sparse_model.to(device)  # move to GPU if possible
sparse_model.eval()

Downloading:   0%|          | 0.00/670 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/418M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [9]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(sparse_model_id)

tokens = tokenizer(data[0]['synoniem'], return_tensors='pt')

In [11]:
with torch.no_grad():
    sparse_emb = sparse_model(
        d_kwargs=tokens.to(device)
    )['d_rep'].squeeze()
sparse_emb.shape

torch.Size([30522])

In [12]:
sparse_emb


tensor([0., 0., 0.,  ..., 0., 0., 0.], device='cuda:0')

In [13]:
indices = sparse_emb.nonzero().squeeze().cpu().tolist()
print(len(indices))

67


In [14]:
values = sparse_emb[indices].cpu().tolist()
sparse = {'indices': indices, 'values': values}
sparse

{'indices': [1012,
  1037,
  2155,
  2160,
  2188,
  2231,
  2282,
  2311,
  2508,
  2534,
  2542,
  2563,
  2577,
  2711,
  2726,
  2773,
  2888,
  3200,
  3235,
  3252,
  3392,
  3506,
  3533,
  3574,
  3776,
  3782,
  3829,
  3847,
  4079,
  4080,
  4205,
  4398,
  4489,
  4545,
  4706,
  4913,
  5014,
  5146,
  5215,
  5899,
  5954,
  5965,
  6108,
  6210,
  6213,
  6330,
  6425,
  6469,
  6609,
  6754,
  6952,
  7010,
  7063,
  7084,
  7330,
  8404,
  8640,
  8981,
  9894,
  10267,
  10505,
  11011,
  14383,
  14473,
  16155,
  18801,
  25209],
 'values': [0.13414156436920166,
  0.4926934838294983,
  0.29473719000816345,
  2.762281894683838,
  2.576303005218506,
  0.12470348179340363,
  0.007782140281051397,
  0.6613984704017639,
  0.025073638185858727,
  0.21922965347766876,
  0.5232481360435486,
  0.23092569410800934,
  0.34370049834251404,
  0.1693800985813141,
  0.6583697199821472,
  0.24631060659885406,
  0.08961215615272522,
  0.4932899475097656,
  0.10291612148284912,
  0.0

In [16]:
idx2token = {idx: token for token, idx in tokenizer.get_vocab().items()}

In [17]:
sparse_dict_tokens = {
    idx2token[idx]: round(weight, 2) for idx, weight in zip(indices, values)
}
# sort so we can see most relevant tokens first
sparse_dict_tokens = {
    k: v for k, v in sorted(
        sparse_dict_tokens.items(),
        key=lambda item: item[1],
        reverse=True
    )
}
sparse_dict_tokens

{'house': 2.76,
 'home': 2.58,
 'houses': 1.49,
 'homes': 1.29,
 'housing': 0.68,
 'building': 0.66,
 'thomas': 0.66,
 'benjamin': 0.66,
 'definition': 0.61,
 'meaning': 0.59,
 'craig': 0.59,
 'madison': 0.55,
 'adam': 0.54,
 'living': 0.52,
 'a': 0.49,
 'property': 0.49,
 'household': 0.42,
 'stewart': 0.42,
 'jacob': 0.41,
 'kitchen': 0.39,
 'ike': 0.37,
 'george': 0.34,
 'helen': 0.34,
 'simon': 0.33,
 'dom': 0.33,
 'geography': 0.31,
 'family': 0.29,
 'estate': 0.29,
 'manor': 0.27,
 'difference': 0.26,
 'word': 0.25,
 'greg': 0.24,
 'england': 0.23,
 'hall': 0.22,
 'burke': 0.21,
 'bradley': 0.2,
 'dave': 0.19,
 'spencer': 0.18,
 'recreation': 0.18,
 'sherman': 0.18,
 'person': 0.17,
 'austin': 0.17,
 '.': 0.13,
 'government': 0.12,
 'eden': 0.12,
 'martin': 0.1,
 'apartment': 0.1,
 'jay': 0.1,
 'handy': 0.1,
 'henry': 0.09,
 'fred': 0.09,
 'homestead': 0.09,
 'mansion': 0.08,
 'clark': 0.07,
 'foster': 0.07,
 'terry': 0.07,
 'joe': 0.04,
 'chris': 0.04,
 'literary': 0.04,
 'james

In [ ]:
from pinecone import Pinecone


def builder(records: list):
    ids = [x['id'] for x in records]
    contexts = [x['synoniem'] for x in records]
    # create dense vecs
    dense_vecs = dense_model.encode(contexts).tolist()
    # create sparse vecs
    input_ids = tokenizer(
        contexts, return_tensors='pt',
        padding=True, truncation=True
    )
    with torch.no_grad():
        sparse_vecs = sparse_model(
            d_kwargs=input_ids.to(device)
        )['d_rep'].squeeze()
    # convert to upsert format
    upserts = []
    for _id, dense_vec, sparse_vec, context in zip(ids, dense_vecs, sparse_vecs, contexts):
        # extract columns where there are non-zero weights
        indices = sparse_vec.nonzero().squeeze().cpu().tolist()  # positions
        values = sparse_vec[indices].cpu().tolist()  # weights/scores
        # build sparse values dictionary
        sparse_values = {
            "indices": indices,
            "values": values
        }
        # build metadata struct
        metadata = {'synoniem': synoniem}
        # append all to upserts list as pinecone.Vector (or GRPCVector)
        upserts.append({
            'id': _id,
            'values': dense_vec,
            'sparse_values': sparse_values,
            'metadata': metadata
        })
    return upserts

In [ ]:
builder(data[:3])

/usr/local/lib/python3.10/dist-packages/torch/amp/autocast_mode.py:250: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(


[{'id': '1-0',
  'values': [0.7600561380386353,
   0.19207192957401276,
   0.41358160972595215,
   -0.4267725348472595,
   0.5056126713752747,
   0.20919199287891388,
   0.14213325083255768,
   -0.19061295688152313,
   -0.2613736093044281,
   -0.4307633936405182,
   -0.3389834761619568,
   -0.169939786195755,
   -0.18128547072410583,
   0.3053555190563202,
   -0.1561288982629776,
   0.06925889104604721,
   0.02423853985965252,
   0.5068425536155701,
   -0.12734107673168182,
   -0.23031136393547058,
   -0.028050342574715614,
   -0.23416881263256073,
   0.18815475702285767,
   -0.03789066523313522,
   0.18597514927387238,
   -0.21104200184345245,
   -0.22678658366203308,
   -0.06973559409379959,
   -0.05393185093998909,
   0.09158752113580704,
   0.16677038371562958,
   -0.23354089260101318,
   -0.03929300233721733,
   -0.3567129075527191,
   0.010374234057962894,
   -0.3489167392253876,
   -0.3755171298980713,
   -0.10844907909631729,
   -0.05983312800526619,
   -0.11230890452861786,
  

In [ ]:
import os
from pinecone import Pinecone

# initialize connection to pinecone (get API key at app.pinecone.io)
api_key = os.environ.get('PINECONE_API_KEY') or '56f6a8f9-46f3-4d07-bd13-6ae5ab8395a4'

# configure client
pc = Pinecone(api_key=api_key)

In [ ]:
from pinecone import ServerlessSpec

cloud = os.environ.get('PINECONE_CLOUD') or 'aws'
region = os.environ.get('PINECONE_REGION') or 'us-east-1'

spec = ServerlessSpec(cloud=cloud, region=region)

In [ ]:
index_name = 'test'

In [ ]:
import time

# check if index already exists (it shouldn't if this is first time)
if index_name not in pc.list_indexes().names():
    # if does not exist, create index
    pc.create_index(
        index_name,
        dimension=dim,
        metric='dotproduct',
        spec=spec
    )
    # wait for index to be initialized
    while not pc.describe_index(index_name).status['ready']:
        time.sleep(1)

# connect to index
index = pc.Index(index_name)
# view index stats
index.describe_index_stats()

{'dimension': 768,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [ ]:
index.upsert(builder(data[:3]))

{'upserted_count': 3}

In [ ]:
from tqdm.auto import tqdm

batch_size = 64

for i in tqdm(range(0, len(data), batch_size)):
    # extract batch of data
    i_end = min(i+batch_size, len(data))
    batch = data[i:i_end]
    # pass data to builder and upsert
    index.upsert(builder(data[i:i+batch_size]))

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
len(data), index.describe_index_stats()


(5,
 {'dimension': 768,
  'index_fullness': 0.0,
  'namespaces': {'': {'vector_count': 5}},
  'total_vector_count': 5})

In [ ]:
def encode(word: str):
    # Retrieve the corresponding row from the DataFrame where 'word1' or 'word2' matches the input word
    row = data_df[(data_df['word1'] == word) | (data_df['word2'] == word)].iloc[0]

    # Extract the synonym from the row
    synonym = row['word1'] if row['word2'] == word else row['word2']

    # Return the synonym
    return synonym

# Example usage
word = "house"
synonym = encode(word)
print(f"The synonym for '{word}' is '{synonym}'")


The synonym for 'house' is 'home'
